In [1]:
import joblib,re,os
import numpy as np
import pandas as pd
import collections
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_pickle('data/amazon_1h.pkl')

In [4]:
df.head()

,categories,train/test,text
0000025240,"[books, new, used & rental textbooks, social s...",train,alan a . block is emeritus professor of crime ...
0001004743,"[books, children's books, classics]",train,grade 4-8 ? the law of the jungle ? intense co...
0001024043,"[books, literature & fiction]",train,trotter street is alive with activity ! mae pr...
0001042335,"[books, new, used & rental textbooks, humaniti...",train,william shakespeare is widely regarded as the ...
0001046349,"[books, literature & fiction, contemporary]",train,"`` no matter how appalling , malicious and pre..."


In [7]:
df['cat_len'] = df['categories'].apply(len)

In [8]:
df['cat_len'].describe()

count    1.493021e+06
mean     3.232762e+00
std      1.213679e+00
min      1.000000e+00
25%      2.000000e+00
50%      3.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: cat_len, dtype: float64

In [28]:
np.mean(df['cat_len'].values>=3)

0.6384973821533656

In [29]:
new_df = df[df['cat_len']>=3]

In [37]:
orig_d = collections.defaultdict(int)
for cat in df['categories'].values:
    for i,c in enumerate(cat):
        orig_d[c]+=1
len(orig_d),len(df)

(11741, 1493021)

In [63]:
cut_line= 4
new_df = df[df['cat_len']>=cut_line]
new_df['categories'] = new_df['categories'].apply(lambda x: x[:cut_line])
new_df['cat_len'] = new_df['categories'].apply(len)
d = collections.defaultdict(int)
for cat in new_df['categories'].values:
    for i,c in enumerate(cat):
        d[c]+=1

In [64]:
print(len(d),len(new_df))
new_df['train/test'].value_counts()

6881 685633


train    542052
test     143581
Name: train/test, dtype: int64

In [65]:
a = set([' -> '.join(cat) for cat in new_df['categories'].values])
len(a)

7544

In [58]:
df['train/test'].value_counts()

train    1186239
test      306782
Name: train/test, dtype: int64

,categories,train/test,text,cat_len,FT
0000025240,"[books, new, used & rental textbooks, social s...",train,alan a . block is emeritus professor of crime ...,5,__label__books->new->used_&_rental_textbooks->...
0001004743,"[books, children's books, classics]",train,grade 4-8 ? the law of the jungle ? intense co...,3,__label__books->children's_books->classics gra...
0001024043,"[books, literature & fiction]",train,trotter street is alive with activity ! mae pr...,2,__label__books->literature_&_fiction trotter s...
0001042335,"[books, new, used & rental textbooks, humaniti...",train,william shakespeare is widely regarded as the ...,5,__label__books->new->used_&_rental_textbooks->...
0001046349,"[books, literature & fiction, contemporary]",train,"`` no matter how appalling , malicious and pre...",3,__label__books->literature_&_fiction->contempo...


In [82]:
a = set([' -> '.join(cat[:min(len(cat),4)]) for cat in df['categories'].values])
len(a)

10239

In [84]:
d = collections.defaultdict(int)
for cats in list(a):
    for cat in cats.split(' -> '):
        d[cat]+=1

In [86]:
len(d)

7883

# remove lower and create new dataset

In [2]:
df = pd.read_pickle('data/amazon_1h.pkl')
cap_at = 3

In [33]:
df.head()

,categories,train/test,text,cat1,cat0,cat2
0000025240,"[books, new, used & rental textbooks, social s...",train,alan a . block is emeritus professor of crime ...,new,books,used & rental textbooks
0001004743,"[books, children's books, classics]",train,grade 4-8 ? the law of the jungle ? intense co...,children's books,books,classics
0001042335,"[books, new, used & rental textbooks, humaniti...",train,william shakespeare is widely regarded as the ...,new,books,used & rental textbooks
0001046349,"[books, literature & fiction, contemporary]",train,"`` no matter how appalling , malicious and pre...",literature & fiction,books,contemporary
0001046438,"[books, literature & fiction, contemporary]",train,"fry is a british polymath -- actor , journalis...",literature & fiction,books,contemporary


In [32]:
df = df[df['categories'].apply(len)>=cap_at]

In [42]:
def do_this(x,n):
    if n>=len(x):
        return ['<NONE>']
    else:
        return [x[n]]

In [43]:

for n in range(cap_at):
    df['cat{}'.format(n)] = df['categories'].apply(lambda x:do_this(x,n))

In [44]:
df.head()

,categories,train/test,text,cat1,cat0,cat2
0000025240,"[books, new, used & rental textbooks, social s...",train,alan a . block is emeritus professor of crime ...,[new],[books],[used & rental textbooks]
0001004743,"[books, children's books, classics]",train,grade 4-8 ? the law of the jungle ? intense co...,[children's books],[books],[classics]
0001042335,"[books, new, used & rental textbooks, humaniti...",train,william shakespeare is widely regarded as the ...,[new],[books],[used & rental textbooks]
0001046349,"[books, literature & fiction, contemporary]",train,"`` no matter how appalling , malicious and pre...",[literature & fiction],[books],[contemporary]
0001046438,"[books, literature & fiction, contemporary]",train,"fry is a british polymath -- actor , journalis...",[literature & fiction],[books],[contemporary]


In [47]:
df[['cat0','cat1','cat2']].describe()

TypeError: unhashable type: 'list'

In [39]:
len(df)

953290

In [48]:
df.head()

,categories,train/test,text,cat1,cat0,cat2
0000025240,"[books, new, used & rental textbooks, social s...",train,alan a . block is emeritus professor of crime ...,[new],[books],[used & rental textbooks]
0001004743,"[books, children's books, classics]",train,grade 4-8 ? the law of the jungle ? intense co...,[children's books],[books],[classics]
0001042335,"[books, new, used & rental textbooks, humaniti...",train,william shakespeare is widely regarded as the ...,[new],[books],[used & rental textbooks]
0001046349,"[books, literature & fiction, contemporary]",train,"`` no matter how appalling , malicious and pre...",[literature & fiction],[books],[contemporary]
0001046438,"[books, literature & fiction, contemporary]",train,"fry is a british polymath -- actor , journalis...",[literature & fiction],[books],[contemporary]


In [49]:
df.to_pickle('data/amazon_1h_trunc3.pkl')